In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Test

In [ ]:
%%time
# arrow_filename = "/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/consensus/consensus-8-of-20.arrow"
arrow_filename = "/home/jqs1/scratch/sequencing/240111_pLIB442-447_revio/output/default/prepare_consensus/consensus-107-of-120.arrow"
df_test = pl.read_ipc(arrow_filename)

In [ ]:
len(df_test)

In [ ]:
df_test.head()

In [ ]:
%%time
df_test.with_columns(
    path_hash2=pl.col("path").hash() % 120,
    path_hash3=processing.categorical_list_hash(pl.col("path")) % 120,
    path_hash4=pl.col("path").list.eval(pl.element().hash()).hash() % 120,
    path_hash5=pl.col("path")
    .cast(pl.List(pl.String))
    .list.eval(pl.element().hash())
    .hash()
    % 120,
)

In [ ]:
%%time
df_test.with_columns(test=pl.col("path").list.eval(pl.element().hash()).hash())

In [ ]:
df_test.with_columns(
    test=pl.col("path").to_physical().list.eval(pl.element().hash()).hash()
)

In [ ]:
%%time
df_test.with_columns(
    test=pl.col("path").cast(pl.List(pl.String)).list.eval(pl.element().hash()).hash()
)

In [ ]:
df_test.head().select(pl.col("path").hash(0))

In [ ]:
df_test.select(pl.col("path").hash() % 120)

In [ ]:
x = df_test["path"].value_counts(sort=True)

In [ ]:
x[0, 0]

In [ ]:
x[1, 0]

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/240111_pLIB442-447_revio/output/default/prepare_consensus/*.arrow"
df_test = pl.read_ipc(arrow_filename)

# 240111_bcd_rbses_revio

In [ ]:
arrow_filename = "/home/jqs1/scratch/sequencing/240111_pLIB442-447_revio/output/default/consensus/*.arrow"
df_revio = pl.read_ipc(arrow_filename)

In [ ]:
df_revio.with_columns(len=pl.col("consensus_seq").str.len_bytes())

In [ ]:
df_revio.select(
    pl.col("grouping_depth").sum().alias("foo")
).to_series()  # .to_dict(as_series=False)

In [ ]:
df_revio["grouping_depth"].to_pandas().hvplot.hist(bins=100, steps)

In [ ]:
x = pl.read_ipc(
    "/n/scratch/users/j/jqs1/sequencing/240111_pLIB442-447_revio/work/52/122a8a96d2c37f2d05a049b7092895/*.arrow"
)

In [ ]:
x.sort("consensus_depth")

In [ ]:
x

# 231207_pLIB442-447

In [ ]:
%%time
arrow_filename = "/home/jqs1/scratch/sequencing/231207_pLIB442-447/20231207_1151_3C_PAU07761_c6097b3e/consensus/*.arrow"
df_ont = pl.read_ipc(arrow_filename)

# Pairwise alignment

In [ ]:
%%time
df_joined = df_revio.with_columns(path_hash=pl.col("path").hash()).join(
    df_ont.with_columns(path_hash=pl.col("path").hash()),
    on="path_hash",
    how="outer_coalesce",
)

In [ ]:
%%time
df_joined = df_revio.with_columns(
    path_hash=pl.col("path").cast(pl.List(pl.String)).list.join("+")
).join(
    df_ont.with_columns(
        path_hash=pl.col("path").cast(pl.List(pl.String)).list.join("+")
    ),
    on="path_hash",
    how="inner",
    validate="1:1",
)

In [ ]:
df_ont["path"].value_counts(sort=True)

In [ ]:
df_joined["path_hash"].value_counts(sort=True)

In [ ]:
len(df_joined)

In [ ]:
df_joined.group_by(
    pl.col("path").is_not_null(), pl.col("path_right").is_not_null()
).agg(pl.len()).with_columns(frac=pl.col("len") / pl.col("len").sum()).rename(
    {"path": "in_revio", "path_right": "in_ont"}
)

In [ ]:
len(df_revio)

In [ ]:
len(df_ont)